In [391]:
path_hcba = 's3n://ewe-meta-top/hcba_data_2015_16/*'
hcba = sqlCtx.read.format("com.databricks.spark.csv").options(header = True, inferschema = True, delimiter = ",",nullValue= 
'\\N')\
    .load(path_hcba)
        

In [392]:
train_end_date = "2016-06-25"
twoweek_start_date  = "2016-06-12"

In [393]:
hcba1 = hcba.drop("")

In [394]:
hcba2 = sqlCtx.read.parquet("s3n://ewe-meta-top/hcba_final_last_yr_may_july")
hcba3 = hcba2.select("HOTEL_ID","PARTNER_POS","DEVICE_TYPE","PARTNER_ORG","ACTIVITY_DATE","TRAFFIC","BOOKING_WINDOW","LOS","COST","GROSS_PROFIT",
                     "LOS_BUCKET","BW_BUCKET","BOOKING","region","MODEL_NAME","STAR_RATING_BUCKET","HOTEL_TYPE_BUCKET","MARKET_BUCKET",\
                     "REGION_BUCKET","CITY_BUCKET",)

In [395]:
hcba4 = hcba1.unionAll(hcba3).repartition(1)

In [396]:
hcba_rm_gp0_book0 = hcba4.filter((hcba4.GROSS_PROFIT !=0.00) & (hcba4.BOOKING != 0) & \
                                 (hcba4.ACTIVITY_DATE <= train_end_date) & (hcba4.DEVICE_TYPE=='DESKTOP') ).cache()

### Importing the data for all POS except CA

In [397]:
hcba_all1 = sqlCtx.read.parquet("s3n://ewe-meta-top/hcba_final_last_yr_may_march_allPOS")
hcba_all2 = hcba_all1.unionAll(sqlCtx.read.parquet("s3n://ewe-meta-top/hcba_final_this_yr_apr_june_allPOS1"))
hcba_all = hcba_all2.select("HOTEL_ID","PARTNER_POS","DEVICE_TYPE","PARTNER_ORG","ACTIVITY_DATE","TRAFFIC","BOOKING_WINDOW","LOS","COST","GROSS_PROFIT",
                     "LOS_BUCKET","BW_BUCKET","BOOKING","region","MODEL_NAME","STAR_RATING_BUCKET","HOTEL_TYPE_BUCKET","MARKET_BUCKET",\
                     "REGION_BUCKET","CITY_BUCKET")

In [398]:
hcba_all_rm = (hcba_all.filter((hcba_all.GROSS_PROFIT !=0.00) & (hcba_all.BOOKING != 0) & \
                                 (hcba_all.ACTIVITY_DATE <= train_end_date) & (hcba_all.DEVICE_TYPE=='DESKTOP') )
               .repartition(1)
               .cache()
               )

### EPB using recent 14 days and agg at hotel_id , los  and bw level

In [399]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE <= train_end_date) & (hcba_rm_gp0_book0.ACTIVITY_DATE >= twoweek_start_date)).\
                groupBy("HOTEL_ID","BW_BUCKET","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [400]:
from pyspark.sql.functions import *
hcba_epb1 = test2.withColumn("EPB1",round("epb",2)).drop("epb").repartition(1).cache()

In [401]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE <= train_end_date) & (hcba_all_rm.ACTIVITY_DATE >=twoweek_start_date)).\
                groupBy("HOTEL_ID","BW_BUCKET","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [402]:
hcba_epb_all1 = test2.withColumn("EPB_all1",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using data before 14 days and only this year and agg at hotel_id , los and bw level

In [403]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE < twoweek_start_date) & (hcba_rm_gp0_book0.ACTIVITY_DATE>='2016-01-01')).\
                groupBy("HOTEL_ID","BW_BUCKET","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [404]:
hcba_epb2 = test2.withColumn("EPB2",round("epb",2)).drop("epb").repartition(1).cache()

In [405]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE < twoweek_start_date) & (hcba_all_rm.ACTIVITY_DATE>='2016-01-01')).\
                groupBy("HOTEL_ID","BW_BUCKET","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [406]:
hcba_epb_all2 = test2.withColumn("EPB_all2",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using recent 14 days data and agg at hotel_id and los level

In [407]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE <= train_end_date) & (hcba_rm_gp0_book0.ACTIVITY_DATE >=twoweek_start_date)).\
                groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [408]:
hcba_epb3 = test2.withColumn("EPB3",round("epb",2)).drop("epb").repartition(1).cache()

In [409]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE <= train_end_date) & (hcba_all_rm.ACTIVITY_DATE >=twoweek_start_date)).\
                groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [410]:
hcba_epb_all3 = test2.withColumn("EPB_all3",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using data before 14 days and only this year and agg at hotel_id and los level

In [411]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE < twoweek_start_date) & (hcba_rm_gp0_book0.ACTIVITY_DATE>='2016-01-01')).\
                groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [412]:
hcba_epb4 = test2.withColumn("EPB4",round("epb",2)).drop("epb").repartition(1).cache()

In [413]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE < twoweek_start_date) & (hcba_all_rm.ACTIVITY_DATE>='2016-01-01')).\
                groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [414]:
hcba_epb_all4 = test2.withColumn("EPB_all4",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using  data of last year and agg at hotel_id ,los and bw level

In [415]:
test1 = hcba_rm_gp0_book0.filter(hcba_rm_gp0_book0.ACTIVITY_DATE < "2016-01-01").\
                groupBy("HOTEL_ID","LOS_BUCKET","BW_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [416]:
hcba_epb5 = test2.withColumn("EPB5",round("epb",2)).drop("epb").repartition(1).cache()

In [417]:
test1 = hcba_all_rm.filter(hcba_all_rm.ACTIVITY_DATE < "2016-01-01").\
                groupBy("HOTEL_ID","LOS_BUCKET","BW_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [418]:
hcba_epb_all5 = test2.withColumn("EPB_all5",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using data of last year and agg at hotel_id and los level

In [419]:
test1 = hcba_rm_gp0_book0.filter(hcba_rm_gp0_book0.ACTIVITY_DATE < "2016-01-01").\
                groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [420]:
hcba_epb51 = test2.withColumn("EPB51",round("epb",2)).drop("epb").repartition(1).cache()

In [421]:
test1 = hcba_all_rm.filter(hcba_all_rm.ACTIVITY_DATE < "2016-01-01").\
                groupBy("HOTEL_ID","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [422]:
hcba_epb_all51 = test2.withColumn("EPB_all51",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using this year data and agg at hotel_id level

In [423]:
test1 = hcba_rm_gp0_book0.filter((hcba_rm_gp0_book0.ACTIVITY_DATE >= '2016-01-01') & (hcba_rm_gp0_book0.ACTIVITY_DATE <= train_end_date)).\
                groupBy("HOTEL_ID").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [424]:
hcba_epb6 = test2.withColumn("EPB6",round("epb",2)).drop("epb").repartition(1).cache()

In [425]:
test1 = hcba_all_rm.filter((hcba_all_rm.ACTIVITY_DATE >= '2016-01-01') & (hcba_all_rm.ACTIVITY_DATE <= train_end_date)).\
                groupBy("HOTEL_ID").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [426]:
hcba_epb_all6 = test2.withColumn("EPB_all6",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at hotel_id level

In [427]:
test1 = hcba_rm_gp0_book0.filter(hcba_rm_gp0_book0.ACTIVITY_DATE < "2016-01-01").\
                groupBy("HOTEL_ID").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [428]:
hcba_epb7 = test2.withColumn("EPB7",round("epb",2)).drop("epb").repartition(1).cache()

In [429]:
test1 = hcba_all_rm.filter(hcba_all_rm.ACTIVITY_DATE < "2016-01-01").\
                groupBy("HOTEL_ID").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [430]:
hcba_epb_all7 = test2.withColumn("EPB_all7",round("epb",2)).drop("epb").repartition(1).cache()

## joining with hotel_dim table to get market and other fields

In [431]:
path_hotel_dim = 's3n://ewe-core-meta-prod/CORE/HOTEL_DIM/*'
hotel_dim = sqlCtx.read.format("com.databricks.spark.csv").options(header = False, inferSchema = True, delimiter = "\t", nullValue= '\\N').			load(path_hotel_dim)
hotel_dim1 = hotel_dim.distinct().filter("C12 = 'BOOKABLE' or C12 ='NOT BOOKABLE' or C12 ='NOT APPLICABLE'").\
                select("C0","C7","C25","C26","C4","C13","C3")
hotel_dim4 = hotel_dim1.withColumnRenamed("C0","hotel_id").withColumnRenamed("C7" , "city").withColumnRenamed("C25","region").\
    withColumnRenamed("C26","market").withColumnRenamed("C4","brand").withColumnRenamed("C13","star_rating").\
    withColumnRenamed("C3","hotel_type")

#removing the records with hotel id 0
hotel_dim5 = hotel_dim4.filter((hotel_dim4['hotel_id'].isNotNull())).withColumnRenamed('hotel_id','HOTEL_ID').repartition(1).cache()


In [432]:
hcba_rm_gp0_book0_dim = hotel_dim5.join(broadcast(hcba_rm_gp0_book0),"HOTEL_ID","inner")

In [433]:
hcba_all_rm_dim = hcba_all_rm.join(hotel_dim5,"HOTEL_ID","inner").repartition(1).cache()

### EPB using this year data and agg at Market,Star_Rating,Hotel_Type,los bucket level

In [434]:
test1 = hcba_rm_gp0_book0_dim.filter((hcba_rm_gp0_book0_dim.ACTIVITY_DATE >= '2016-01-01') & (hcba_rm_gp0_book0_dim.ACTIVITY_DATE <= train_end_date)).\
                groupBy("market","star_rating","hotel_type","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [435]:
hcba_epb8 = test2.withColumn("EPB8",round("epb",2)).drop("epb").repartition(1).cache()

In [436]:
test1 = hcba_all_rm_dim.filter((hcba_all_rm_dim.ACTIVITY_DATE >= '2016-01-01' ) & (hcba_all_rm_dim.ACTIVITY_DATE <= train_end_date) ).\
                groupBy("market","star_rating","hotel_type","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [437]:
hcba_epb_all8 = test2.withColumn("EPB_all8",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at Market,Star_Rating,Hotel_Type,los bucket level

In [438]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE < '2016-01-01'  ").\
                groupBy("market","star_rating","hotel_type","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [439]:
hcba_epb9 = test2.withColumn("EPB9",round("epb",2)).drop("epb").repartition(1).cache()

In [440]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE < '2016-01-01'  ").\
                groupBy("market","star_rating","hotel_type","LOS_BUCKET").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [441]:
hcba_epb_all9 = test2.withColumn("EPB_all9",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using this year data and agg at Market,Star_Rating,Hotel_Type level

In [442]:
test1 = hcba_rm_gp0_book0_dim.filter((hcba_rm_gp0_book0_dim.ACTIVITY_DATE >= '2016-01-01' ) & (hcba_rm_gp0_book0_dim.ACTIVITY_DATE <= train_end_date) ).\
                groupBy("market","star_rating","hotel_type").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [443]:
hcba_epb10 = test2.withColumn("EPB10",round("epb",2)).drop("epb").repartition(1).cache()

In [444]:
test1 = hcba_all_rm_dim.filter((hcba_all_rm_dim.ACTIVITY_DATE >= '2016-01-01' ) & (hcba_all_rm_dim.ACTIVITY_DATE <= train_end_date) ).\
                groupBy("market","star_rating","hotel_type").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [445]:
hcba_epb_all10 = test2.withColumn("EPB_all10",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at Market,Star_Rating,Hotel_Type level

In [446]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE < '2016-01-01'  ").\
                groupBy("market","star_rating","hotel_type").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [447]:
hcba_epb11 = test2.withColumn("EPB11",round("epb",2)).drop("epb").repartition(1).cache()

In [448]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE < '2016-01-01'  ").\
                groupBy("market","star_rating","hotel_type").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [449]:
hcba_epb_all11 = test2.withColumn("EPB_all11",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using this year data and agg at Market level

In [450]:
test1 = hcba_rm_gp0_book0_dim.filter((hcba_rm_gp0_book0_dim.ACTIVITY_DATE >= '2016-01-01' ) & (hcba_rm_gp0_book0_dim.ACTIVITY_DATE <= train_end_date) ).\
                groupBy("market").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [451]:
hcba_epb12 = test2.withColumn("EPB12",round("epb",2)).drop("epb").repartition(1).cache()

In [452]:
test1 = hcba_all_rm_dim.filter((hcba_all_rm_dim.ACTIVITY_DATE >= '2016-01-01' ) & (hcba_all_rm_dim.ACTIVITY_DATE <= train_end_date) ).\
                groupBy("market").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [453]:
hcba_epb_all12 = test2.withColumn("EPB_all12",round("epb",2)).drop("epb").repartition(1).cache()

### EPB using last year data and agg at Market level

In [454]:
test1 = hcba_rm_gp0_book0_dim.filter("ACTIVITY_DATE < '2016-01-01'  ").\
                groupBy("market").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [455]:
hcba_epb13 = test2.withColumn("EPB13",round("epb",2)).drop("epb").repartition(1).cache()

In [456]:
test1 = hcba_all_rm_dim.filter("ACTIVITY_DATE < '2016-01-01'  ").\
                groupBy("market").agg({"GROSS_PROFIT":"sum","BOOKING":"sum"})
test2 = test1.withColumn("epb",(test1["sum(GROSS_PROFIT)"]/test1["sum(BOOKING)"])).drop("sum(BOOKING)").drop("sum(GROSS_PROFIT)")

In [457]:
hcba_epb_all13 = test2.withColumn("EPB_all13",round("epb",2)).drop("epb").repartition(1).cache()

### creating universe for optimization

In [458]:
from datetime import timedelta, date
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import *
import time

##fucntion for chaging the datatype
def data_type_change_int(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(IntegerType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

def data_type_change_float(df,cols):
    temp = df
    for col in cols:
        temp = temp.withColumn(col+'_1',df[col].cast(DoubleType())).drop(col).withColumnRenamed(col+'_1',col)
    return temp

#star rating buckets
def srch_region_star_rating(star_rating):
    if star_rating == '2.0':
        return '2'
    elif star_rating == '2.5':
        return '2.5'
    elif star_rating == '3.0':
        return '3'
    elif star_rating == '3.5':
        return '3.5'
    elif star_rating == '4.0':
        return '4'
    elif star_rating == '1.0' or star_rating == '1.5':
        return '1-1.5'
    elif star_rating == '4.5' or star_rating == '5.0':
        return '4.5-5'
    else:
        return '0'

#udf for creating city buckets
def srch_region_city(city,model_name):
	if model_name == 'CANADA':
		if city in ('TORONTO' , 'NIAGARA FALLS' , 'MONTREAL' , 'VANCOUVER' , 'QUEBEC' , 'BANFF' , 'OTTAWA' , 'EDMONTON' , 'VICTORIA','CALGARY' , 'WHISTLER'):
			return city
		elif city in ('NIAGARA-ON-THE-LAKE','CANMORE','JASPER','KELOWNA','HALIFAX','MISSISSAUGA','LAKELOUISE','COLLINGWOOD','VERNON','PENTICTON','GATINEAU','DORVAL','MARKHAM'):
			return'G12'
		elif city in ('MONT-TREMBLANT','TOFINO','BLUEMOUNTAINS','OSOYOOS','UCLUELET','SAINT-SAUVEUR','WESTKELOWNA','LAMALBAIE','SAINTE-ADELE'):
			return'G13'
		elif city in ('WINNIPEG','LONDON','RICHMOND','BARRIE','REVELSTOKE','VAUGHAN','ORILLIA','BURNABY','SURREY'):
			return 'G14'
		else:
			return 'others'
	else:
		return 'NA'

#udf for hotel type
def srch_region_Hotel_type(Hotel_type,model_name):
	if model_name == 'CANADA':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' , 'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL', 'INN'):
			return 'Family_Premium'
		else:
			return 'Others'
	
	elif model_name == 'US':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'MOTEL'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION' , 'CABIN' , 'GUEST HOUSE'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,  'INN' , 'APARTMENT', 'VILLA', 'PRIVATE VACATION HOME'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Mexico':
		if Hotel_type in  ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE'):
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('LODGE' , 'BED & BREAKFAST' , 'HOSTEL/BACKPACKER ACCOMMODATION'):
			return 'Budget_hotel'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'ALL-INCLUSIVE') :
			return Hotel_type
		elif Hotel_type in ('CONDOMINIUM RESORT' ,'CONDO'):
			return 'CONDO'
		elif Hotel_type in ('APART-HOTEL' ,'INN' , 'VILLA' , 'APARTMENT'):
			return 'Family_Premium'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		if Hotel_type in ('HOTEL' , 'HOTEL RESORT' , 'APART-HOTEL') :
			return Hotel_type
		elif Hotel_type in ('GUEST HOUSE' ,'BED & BREAKFAST', 'HOSTEL/BACKPACKER ACCOMMODATION', 'MOTEL' , 'HOSTAL (BUDGET HOTEL)'):
			return 'Budget_hotel'
		elif Hotel_type in ('APARTMENT','RESIDENCE','TOWNHOUSE','INN'):
			return 'Budget_hotel'
		else:
			return 'others'
	
	elif model_name=="North America":
		if Hotel_type in ('HOTEL' , 'MOTEL') :
			return Hotel_type
		elif Hotel_type in ('LODGE' ,  'BED & BREAKFAST' ,  'COTTAGE' ,  'GUEST HOUSE' ,  'CABIN'):
			return 'Budget_hotel'
		elif Hotel_type in ( 'APARTMENT' ,  'HOTEL RESORT' ,  'CONDO' ,  'INN' ,  'APART-HOTEL' ,  'CARAVAN PARK'):
			return 'Family_Premium'
		else:
			return 'others'
	else:
		return 'NOT APPLICABLE'

##udf for market bucket
def srch_region_market(market,model_name):
	if model_name == 'CANADA':
		if market in  ('TORONTO, ON, CAN' , 'NIAGARA FALLS, ON, CAN' , 'MONTREAL, QC, CAN' , 'BANFF AREA, AB, CAN' ,'VANCOUVER, BC, CAN' ,'QUEBEC CITY, QC, CAN' , 'EDMONTON, AB, CAN' , 'OTTAWA, ON, CAN' , 'OKANAGAN VALLEY, BC, CAN'):
			return market
		elif market in ('VICTORIA, BC, CAN' ,'CALGARY, AB, CAN' ,'JASPER AREA, AB, CAN'):
			return 'G10'
		elif market in ('WHISTLER, BC, CAN' ,'NIAGARA-ON-THE-LAKE, ON, CAN' ,'MONT TREMBLANT, QC, CAN' , 'UCLUELET, BC, CAN','TOFINO - UCLUELET, BC, CAN'):
			return 'G11'
		elif market in ('COLLINGWOOD, ON, CAN' , 'BC INTERIOR SKI, BC, CAN' ,'HALIFAX, NS, CAN','WINNIPEG, MB, CAN' ,'LONDON, ON, CAN' , 'KAMLOOPS, BC, CAN'):
			return 'G12'
		else:
			return 'others'
	
	elif model_name == 'US':
		return 'NA'
		
	elif model_name == 'Mexico':
		if market in  ('MEXICO - RIVIERA MAYA, PLAYA DEL CARMEN & TULUM' , 'MEXICO - CANCUN & ISLA MUJERES' ,'MEXICO - PUERTO VALLARTA, NUEVO VALLARTA & RIVIERA NAYARIT' , 'MEXICO - LOS CABOS' , 'CENTRAL AMERICA - COSTA RICA - GUANACASTE' ,'CENTRAL AMERICA - BELIZE' , 'CENTRAL AMERICA - COSTA RICA - PUNTARENAS' , 'CENTRAL AMERICA - COSTA RICA - OTHER' , 'MEXICO - COZUMEL''MEXICO - HUATULCO' , 'MEXICO - IXTAPA & ZIHUATANEJO' , 'CENTRAL AMERICA - HONDURAS' , 'CENTRAL AMERICA - PANAMA - OTHER') :
			return market
		elif market in ( 'CENTRAL AMERICA - COSTA RICA - SAN JOSE' , 'CENTRAL AMERICA - PANAMA - PANAMA CITY' ,'MEXICO - MEXICO CITY' ,'MEXICO - MEXICO CITY'						,'CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR','MEXICO - ACAPULCO'):
			return 'hclt'
		elif market in ('CENTRAL AMERICA - NICARAGUA','MEXICO - MAZATLAN','MEXICO - MANZANILLO','CENTRAL AMERICA - EL SALVADOR',                        'MEXICO - ACAPULCO'):
			return 'lclt'
		else:
			return 'others'
		
	elif model_name=="CARIBBEAN":
		if market in ('PUNTA CANA, DOMINICAN REPUBLIC' , 'TURKS AND CAICOS' , 'MONTEGO BAY, JAMAICA' , 'ST. LUCIA' ,'OCHO RIOS, JAMAICA' ,                      'NEGRIL, JAMAICA' , 'BARBADOS'):
			return market
		elif market in ('ARUBA' , 'PUERTO PLATA, DOMINICAN REPUBLIC' , 'ANTIGUA AND BARBUDA'):
			return 'G8'
		elif market in ('NASSAU, BAHAMAS', 'DOMINICAN REPUBLIC ALL OTHER' , 'BERMUDA','BAHAMAS, OUT ISLANDS'):
			return 'G9'
		elif market in ('SAN JUAN, PUERTO RICO' ,'PARADISE ISLAND, BAHAMAS' ,'CAYMAN ISLANDS','SINT MAARTEN, DUTCH' ,'CURACAO','ST. MARTIN, FRENCH'):
			return 'G10'
		else:
			return 'others'
		
	elif model_name == 'Other Region':
		return 'NA'
	
	elif model_name=="North America":
		if  market in (  'GRAVENHURST - BRACEBRIDGE - HUNTSVILLE, ON, CAN','ONTARIO WEST, ON, CAN','ABBOTSFORD - CHILLIWACK, BC, CAN',                       'MAINE - COAST','ONTARIO EAST, ON, CAN','VANCOUVER ISLAND, BC, CAN','BURLINGTON, VT','CHARLOTTETOWN, PEI, CAN'): 
			return 'G1'
		elif market in ('ALBERTA EAST, AB, CAN','REGINA, SK, CAN','KINGSTON, ON, CAN','SASKATOON, SK, CAN','MONCTON, NB, CAN',                        'TRI-CITIES (KW-GUELPH-CAMBRIDGE), ON, CAN'): 
			return 'G2'
		elif market in ('WINDSOR, ON, CAN','RED DEER, AB, CAN','BRITISH COLUMBIA SOUTH, BC, CAN','GRAND FORKS, ND','SUDBURY, ON, CAN',                        'ONTARIO NORTH, ON, CAN','BANGOR, ME','HAMILTON - BRANTFORD, ON, CAN','BELLEVILLE - TRENTON - COBOURG, ON, CAN','FARGO, ND'): 
			return 'G3'
		elif market in ('LAKE PLACID, NY','PARKSVILLE, VANCOUVER ISLAND, BC, CAN','LAKE GEORGE, NY','OGUNQUIT, ME','EVERETT, WA',                        'COASTAL, BC, CAN','GROVE CITY, PA','BELLINGHAM, WA','NORTH CONWAY, NH','NEWFOUNDLAND OTHER, NL, CAN',                        'NOVA SCOTIA SOUTH, NS, CAN','PORTSMOUTH, NH'): 
			return 'G4'
		elif market in ('LETHBRIDGE, AB, CAN','FREDERICTON, NB, CAN','BRANDON, MB, CAN','ERIE, PA','NANAIMO, VANCOUVER ISLAND, BC, CAN',                        'SASKATCHEWAN SOUTH, SK, CAN','PRINCE GEORGE - QUESNEL, BC, CAN','MEDICINE HAT, AB, CAN','GRANDE PRAIRIE, AB, CAN',                        'ALBERTA NORTH, AB, CAN','DURHAM, ON, CAN','BRITISH COLUMBIA CENTRAL, BC, CAN','ONTARIO SOUTHWEST, ON, CAN',                        'SAULT STE. MARIE, ON, CAN','THUNDER BAY, ON, CAN','SAINT JOHN, NB, CAN','GREAT FALLS, MT','MINOT, ND (AREA)',                        'BRITISH COLUMBIA NORTH, BC, CAN','NORTH BAY, ON, CAN','PETERBOROUGH, ON, CAN','CAPE BRETON ISLAND, NS, CAN','ALBERTA WEST, AB, CAN',                        'HUDSON VALLEY - POUGHKEEPSIE, NY','FORT MCMURRAY, AB, CAN','NOVA SCOTIA NORTH, NS, CAN','WATERTOWN, NY','EDMUNSTON, NB, CAN',                        'OLYMPIC NATIONAL PARK AREA, WA','CEDAR CITY - BRYCE CANYON NATIONAL PARK, UT','SUMMERSIDE, PEI, CAN','SALEM, OR (AREA)','BUTTE - HELENA, MT'                        ,'BECKLEY, WV'): 
			return 'G5'
		elif market in ('SAGINAW, MI','WHITE MOUNTAINS, NH','FINGER LAKES NEW YORK','NEW YORK - WEST','NEW BRUNSWICK NORTH, NB, CAN',                        'SASKATCHEWAN NORTH, SK, CAN','MANITOBA SOUTH, MB, CAN','BAR HARBOR, ME','WASHINGTON NORTHWEST','VERMONT CENTRAL','ALBERTA SOUTH, AB, CAN',                        'WENATCHEE-LEAVENWORTH,WA','GANANOQUE, ON, CAN','PLATTSBURGH, (NY)','STRAITS OF MACKINAC, MI','SANDUSKY, OH','MANCHESTER, NH','ITHACA, NY',                        'MINNESOTA NORTHWEST','MISSOULA, MT (AREA)','YELLOWKNIFE, NT, CAN','PEI OTHER, PEI, CAN','BILLINGS, MT (AREA)','RAPID CITY - MOUNT RUSHMORE, SD',                        'ANDOVER, MA','YUMA, AZ','PORT HURON, MI','LANCASTER, PA','FORT LEE - PARAMUS, NJ','LEXINGTON, KY','MANITOBA NORTH, MB, CAN','ABERDEEN - OCEAN SHORES, WA',                        'CANADA TERRITORIES, YT, CAN','KNOXVILLE, TN','BURLINGTON - MOUNT VERNON, WA','FREDERICKSBURG, VA','ARIZONA SOUTHEAST',                        'WINDHAM COUNTY, VT','KENAI PENINSULA, AK','COLUMBIA, SC','CALIFORNIA DESERTS SOUTH'): 
			return 'G6'
		elif  market in ('ROME-UTICA, NY','SANDPOINT, ID (AREA)','MOAB - GREEN RIVER, UT','WHITEHORSE, YT, CAN','NEW YORK - NORTHEAST',                         'SARNIA, ON, CAN','AUGUSTA, ME','LAKE POWELL - GLEN CANYON NATIONAL PARK, AZ','NEW YORK - SOUTH','BISMARK, ND',                         'ALABAMA NORTH','ASHLAND - MEDFORD, OR','WASHINGTON NORTHEAST','THE BERKSHIRES, MA','EUGENE, OR (AREA)',                         'GRAND RAPIDS, MI','WASHINGTON SOUTHWEST','LAKE HAVASU CITY, AZ','SARATOGA, NY','WYTHEVILLE - VA',                         'ARCATA - EUREKA, CA','SOUTH CAROLINA EAST','CHATTANOOGA, TN','TOLEDO, OH','IDAHO FALLS, ID','WEST VIRGINIA SOUTH',                         'SPRINGFIELD, MA','OLYMPIA, WA','GETTYSBURG - PA'): 
			return 'G7'
		else:return 'Others'
	else:return 'NOT APPLICABLE'
		
def srch_model_name(region):
	if region == 'CANADA':
		return region
		
	elif region in ('FLORIDA (USA)','NEW YORK CITY','CALIFORNIA','GAMING (USA)','NORTHEAST (USA)','MOUNTAIN (USA)',                    'MIDWEST (USA)','CENTRAL (USA)','SF PACNW','ORLANDO','HAWAII'):
		return 'US'
		
	elif region in ("MEXICO - DOMESTIC REGION",'MEXICO - RESORTS REGION','CENTRAL AMERICA REGION'):
		return 'Mexico'
		
	elif region=="CARIBBEAN":
		return 'CARIBBEAN'
    
	elif region=="NORTH AMERICA REGIONAL TERRITORIES":
		return 'North America'
		
	else :
		return "Other Region"

def srch_region_bucket(region,model_name):
	if model_name == 'Other Region':
		if region in ('SPAIN & PORTUGAL' , 'ROME & ITALY RESORTS' , 'MID-ATLANTIC' , 'GREECE & TURKEY' , 'OCEANIA' , 'ITALY NORTH' ,                      'THAILAND' , 'MIDDLE EAST AND INDIAN OCEAN'):
			return region
		elif region in ('PARIS' , 'LONDON' , 'ENGLAND'):
			return 'G9'
		elif region in ('EUROPE REGIONAL TERRITORIES' , 'UK & IRELAND'):
			return 'G10'
		elif region in ('SOUTH AMERICA' , 'AFRICA AND EAST MED'):
			return 'G11'
		elif region in ('EASTERN EUROPE' , 'GERMANY, AUSTRIA & SWITZERLAND' , 'BENELUX' , 'FRANCE' , 'NORDIC'):
			return 'G12'
		elif region in ('JAPAN & MICRONESIA' , 'HONG KONG & MACAU' , 'SOUTHEAST ASIA EMERGING' , 'MAINLAND CHINA'):
			return 'G13'
		else:
			return 'others'
	else :
		return 'NA'


udfsrch_region_star_rating=udf(srch_region_star_rating, StringType())
udfsrch_region_city=udf(srch_region_city, StringType())
udfsrch_region_hotel_type=udf(srch_region_Hotel_type, StringType())
udfsrch_region_market=udf(srch_region_market, StringType())
udfsrch_model_name= udf(srch_model_name,StringType())
udfsrch_region_bucket = udf (srch_region_bucket,StringType())

hotel_dim_model_name = hotel_dim5.withColumn("MODEL_NAME", udfsrch_model_name("region"))
hotel_dim_bkts = (hotel_dim_model_name.withColumn("STAR_RATING_BUCKET", udfsrch_region_star_rating("star_rating")).
                  withColumn("HOTEL_TYPE_BUCKET", udfsrch_region_hotel_type("hotel_type","MODEL_NAME")).    
                  withColumn("MARKET_BUCKET",udfsrch_region_market("market","MODEL_NAME")).     
                  withColumn("REGION_BUCKET", udfsrch_region_bucket("region","MODEL_NAME"))      
                  .withColumn("CITY_BUCKET", udfsrch_region_city("city","MODEL_NAME"))
                  .repartition(1).
                 cache())



In [459]:
bw = ['<=0','1','2','3-4','5-7','8-14','15-20','21-30','31-40','41-60','61-90','91-120','121-180','181-240','>240']
los = ['0','1','2','3','4','5','6','7','8-14','>14']

counter = 1

for i in range(len(los)):
    for j in range(len(bw)):
        if counter == 1:
            sq_univ = hotel_dim_bkts.withColumn("LOS_BUCKET",lit(los[i])).withColumn("BW_BUCKET",lit(bw[j]))
        else : 
            temp = hotel_dim_bkts.withColumn("LOS_BUCKET",lit(los[i])).withColumn("BW_BUCKET",lit(bw[j]))
            sq_univ = sq_univ.unionAll(temp)
            
        counter+=1
        
sq_univ_re = sq_univ.repartition(4).cache()

In [460]:
sq_univ1 = sq_univ_re.join((hcba_epb1),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [461]:
sq_univ2 = sq_univ1.join((hcba_epb2),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [462]:
sq_univ11 = sq_univ2.join((hcba_epb_all1),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [463]:
sq_univ21 = sq_univ11.join((hcba_epb_all2),["HOTEL_ID","BW_BUCKET","LOS_BUCKET"],"left_outer")

In [464]:
sq_univ3 = sq_univ21.join(hcba_epb3,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [465]:
sq_univ4 = sq_univ3.join(hcba_epb4,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [466]:
sq_univ31 = sq_univ4.join(hcba_epb_all3,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [467]:
sq_univ41 = sq_univ31.join(hcba_epb_all4,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [468]:
sq_univ5 = sq_univ41.join(hcba_epb5,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"left_outer")

In [469]:
sq_univ6 = sq_univ5.join(hcba_epb51,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [470]:
sq_univ51 = sq_univ6.join(hcba_epb_all5,["HOTEL_ID","LOS_BUCKET","BW_BUCKET"],"left_outer")

In [471]:
sq_univ61 = sq_univ51.join(hcba_epb_all51,["HOTEL_ID","LOS_BUCKET"],"left_outer")

In [472]:
sq_univ7 = sq_univ61.join(hcba_epb6,["HOTEL_ID"],"left_outer")

In [473]:
sq_univ8 = sq_univ7.join(hcba_epb7,["HOTEL_ID"],"left_outer")

In [474]:
sq_univ71 = sq_univ8.join(hcba_epb_all6,["HOTEL_ID"],"left_outer")

In [475]:
sq_univ81 = sq_univ71.join(hcba_epb_all7,["HOTEL_ID"],"left_outer")

In [476]:
sq_univ9 = sq_univ81.join(hcba_epb8,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [477]:
sq_univ10 = sq_univ9.join(hcba_epb9,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [478]:
sq_univ91 = sq_univ10.join(hcba_epb_all8,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [479]:
sq_univ101 = sq_univ91.join(hcba_epb_all9,["market","star_rating","hotel_type","LOS_BUCKET"],"left_outer")

In [480]:
sq_univ11 = sq_univ101.join(hcba_epb10,["market","star_rating","hotel_type"],"left_outer")

In [481]:
sq_univ12 = sq_univ11.join(hcba_epb11,["market","star_rating","hotel_type"],"left_outer")

In [482]:
sq_univ111 = sq_univ12.join(hcba_epb_all10,["market","star_rating","hotel_type"],"left_outer")

In [483]:
sq_univ121 = sq_univ111.join(hcba_epb_all11,["market","star_rating","hotel_type"],"left_outer")

In [484]:
sq_univ_coal = (sq_univ121.withColumn("EPB",coalesce('EPB1','EPB2','EPB3','EPB4','EPB5','EPB51','EPB6','EPB7','EPB8','EPB9','EPB10','EPB11',
                'EPB_all1','EPB_all2','EPB_all3','EPB_all4','EPB_all5','EPB_all51','EPB_all6','EPB_all7','EPB_all8','EPB_all9',
                                                    'EPB_all10','EPB_all11')))
sq_univ_final= (sq_univ_coal.select([col for col in sq_univ_coal.columns if col not in 
                {'EPB1','EPB2','EPB3','EPB4','EPB5','EPB51','EPB6','EPB7','EPB8','EPB9','EPB10','EPB11',
                'EPB_all1','EPB_all2','EPB_all3','EPB_all4','EPB_all5','EPB_all51','EPB_all6','EPB_all7','EPB_all8','EPB_all9',
                 'EPB_all10','EPB_all1'} ])
               .cache())

In [387]:
sq_univ_final[sq_univ_final.EPB.isNull()].count()

5647200

In [388]:
sq_univ_final.count()

47689350

In [389]:
5647200/47689350

0.11841637598331703

In [485]:
sq_univ_final.write.parquet("s3n://ewe-meta-top/TA_CA_July_4weeks_0826/sq_univ_12Jun_2Jul")

Py4JJavaError: An error occurred while calling o8771.parquet.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelation$$anonfun$run$1.apply$mcV$sp(InsertIntoHadoopFsRelation.scala:156)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelation$$anonfun$run$1.apply(InsertIntoHadoopFsRelation.scala:108)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelation$$anonfun$run$1.apply(InsertIntoHadoopFsRelation.scala:108)
	at org.apache.spark.sql.execution.SQLExecution$.withNewExecutionId(SQLExecution.scala:56)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelation.run(InsertIntoHadoopFsRelation.scala:108)
	at org.apache.spark.sql.execution.ExecutedCommand.sideEffectResult$lzycompute(commands.scala:58)
	at org.apache.spark.sql.execution.ExecutedCommand.sideEffectResult(commands.scala:56)
	at org.apache.spark.sql.execution.ExecutedCommand.doExecute(commands.scala:70)
	at org.apac